In [8]:
import numpy as np
import h5py

def get_nj(file):
    with h5py.File(file, 'r') as f:
        nj = f["INPUTS/Source/MASK"][...].sum(axis=1)
    return nj

def score(ans1, ans2):
    set1 = {frozenset(row) for row in ans1.reshape(3, 2)}
    set2 = {frozenset(row) for row in ans2.reshape(3, 2)}
    return sum([1 for p in set1 if p in set2])

def compare_if_3h(ans_test, ans_pred, nj, nj_target):
    if nj_target == 8:
        mask = (ans_test != -1).all(axis=1) & (nj >= 8)
    else:
        mask = (ans_test != -1).all(axis=1) & (nj == nj_target)

    masked_ans_test = ans_test[mask]
    masked_ans_pred = ans_pred[mask]
    
    compare = np.array([score(masked_ans_test[i], masked_ans_pred[i]) for i in range(len(masked_ans_test))])
    return compare

def get_match_answers(test_file, predict_file): # no sort
    with h5py.File(test_file, "r") as f_test:
        b0_test = f_test["TARGETS/h1/b1"][...]
        b1_test = f_test["TARGETS/h1/b2"][...]
        b2_test = f_test["TARGETS/h2/b1"][...]
        b3_test = f_test["TARGETS/h2/b2"][...]
        b4_test = f_test["TARGETS/h3/b1"][...]
        b5_test = f_test["TARGETS/h3/b2"][...]
    ans_test = np.array((b0_test, b1_test, b2_test, b3_test, b4_test, b5_test)).transpose()
    ans_test[:,0:2].sort(axis=1)
    ans_test[:,2:4].sort(axis=1)
    ans_test[:,4:6].sort(axis=1)

    with h5py.File(predict_file, "r") as f_pred:
        b0_pred = f_pred["SpecialKey.Targets/h1/b1"][...]
        b1_pred = f_pred["SpecialKey.Targets/h1/b2"][...]
        b2_pred = f_pred["SpecialKey.Targets/h2/b1"][...]
        b3_pred = f_pred["SpecialKey.Targets/h2/b2"][...]
        b4_pred = f_pred["SpecialKey.Targets/h3/b1"][...]
        b5_pred = f_pred["SpecialKey.Targets/h3/b2"][...]
    ans_pred = np.array((b0_pred, b1_pred, b2_pred, b3_pred, b4_pred, b5_pred)).transpose()
    ans_pred[:,0:2].sort(axis=1)
    ans_pred[:,2:4].sort(axis=1)
    ans_pred[:,4:6].sort(axis=1)

    return ans_test, ans_pred

In [72]:
file1 = "/home/dilatino/root_to_h5/test/3h_1500_1000_4b_test_27052.h5"
file2 = "/home/dilatino/root_to_h5/test/3h_1500_1000_4b_test_27052_predict_50k.h5"

ans_test, ans_pred = get_match_answers(file1, file2)
nj = get_nj(file1)
count = []
for nj_target in range(6, 9):
    scores = compare_if_3h(ans_test, ans_pred, nj, nj_target)
    count.append([int((scores == i).sum()) for i in range(3, -1, -1)])

count = np.array(count)
count = np.concat((count.sum(axis=1)[:, np.newaxis], count), axis=1)
count = np.concat((count, count.sum(axis=0)[np.newaxis]))
print(count)

rate = np.concat(((count[:, 0] / count[3, 0])[:, np.newaxis], count[:, 1:] / count[:, 0][:,np.newaxis]), axis=1)
rate = np.concat((rate, (np.inner(rate, np.array([0, 3, 2, 1, 0])) / 3)[:, np.newaxis]), axis=1)
print(p)
np.set_printoptions(precision=3)
print(rate)

SyntaxError: invalid syntax (2510321967.py, line 17)

In [3]:
from sklearn.metrics import roc_auc_score, accuracy_score

with h5py.File(file1, 'r') as f:
    signal_test = f["CLASSIFICATIONS/EVENT/signal"][...]        

with h5py.File(file2, 'r') as f:
    signal = f["SpecialKey.Classifications/EVENT/signal"][...]
signal_pred = signal[:,0] < signal[:,1]

acc = accuracy_score(signal_test, signal_pred)
auc = roc_auc_score(signal_test, signal[:, 1])
print(acc)
print(auc)


0.7520700872393908
0.8550971618474714


50k prediction
acc = 0.752
auc = 0.855
